In [1]:
# !pip install PyWavelets
# !pip install -U mediapipe 
# !pip install cohere
# !pip install empath

In [2]:
# !pip install git+https://github.com/openai/whisper.git 
# !conda install -c conda-forge ffmpeg -y

In [3]:
# !pip freeze

In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pywt
import joblib
import mediapipe as mp

In [5]:
# Create a VideoCapture object and read from input file
cap = cv2.VideoCapture('video.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")

In [6]:
findFace = mp.solutions.face_detection.FaceDetection()
def faceBox(frame):
    try:
        frameRGB = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    except:
        return []
    results = findFace.process(frameRGB)
    myFaces = []
    if results.detections != None:
        for face in results.detections:
            bBox = face.location_data.relative_bounding_box
            myFaces.append(bBox)
    return myFaces

def w2d(img, wavelet, level=4):
    imArray = img
    # imArray = cv2.cvtColor(imArray,cv2.COLOR_BGR2GRAY)
    imArray = np.float32(imArray)
    imArray /= 255
    coeffs = pywt.wavedec2(imArray, wavelet, level=level)
    coeffs_H = list(coeffs)
    coeffs_H[0] *= 0
    arrH = pywt.waverec2(coeffs_H, mode)
    arrH += 255
    arrH = np.uint8(arrH)
    return arrH

In [7]:
predicted_values = []; 
model = joblib.load("svm_model.joblib")

def analyse_frame(frame):
    X = []
    scaled_raw_img = cv2.resize(frame, (80,80))
    img_har = w2d(frame,'bior2.2',4)
    scaled_img_har = cv2.resize(img_har, (80,80))
    combined_img = np.vstack((scaled_raw_img.reshape(80*80, 1), scaled_img_har.reshape(80*80*1, 1)))
    X.append(combined_img)
    plt.imshow(img_har, cmap='gray')

    X = np.array(X).reshape(len(X), 12800).astype(float)
    y_pred = model.predict(X)
    predicted_values.append(y_pred)

C:\Users\Ashwin-Rao\anaconda3\envs\music\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Ashwin-Rao\anaconda3\envs\music\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Ashwin-Rao\anaconda3\envs\music\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.5.2 when using version 1.3

In [9]:
def preprocess_frame(frame):
    faceBoxes = faceBox(frame)
    height,width = frame.shape
    x,y,w,h = int(faceBoxes[0].xmin*width),int(faceBoxes[0].ymin*height),int(faceBoxes[0].width*width),int(faceBoxes[0].height*height)
    croppedImage = frame[y:y+h,x:x+w]
    return croppedImage

In [ ]:
# frame = cv2.imread(r"C:\Users\Ashwin-Rao\Downloads\video_proj\20241101_094851.jpg", cv2.IMREAD_GRAYSCALE)
# scaled_raw_img = preprocess_frame(frame)
# analyse_frame(scaled_raw_img)

In [ ]:
# predicted_values

In [ ]:
# if predicted_values[0] == 0:
#         print("Angry:")
# elif predicted_values[0] == 1:
#         print("Disgusted:")
# elif predicted_values[0] == 2:
#         print("Fearful:")
# elif predicted_values[0] == 3:
#         print("Happy:")
# elif predicted_values[0] == 4:
#         print("Neutral:")
# elif predicted_values[0] == 5:
#         print("Sad:")
# elif predicted_values[0] == 6:
#         print("Surprised:")

In [10]:
import whisper

transcription_model = whisper.load_model("base")
result = transcription_model.transcribe("video.mp4")
print(result["text"])

C:\Users\Ashwin-Rao\anaconda3\envs\music\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=de

 A lot of people out there who struggle with depression or even subclinical depression where they don't necessarily have a diagnosis will struggle to pull themselves out of it. They'll analyze their situation and they'll recognize, okay, I need to do this or I need to do this, but they can't seem to despite all the logic bring themselves out of it. And on the flip side, we also have something incredibly scary, which is that sometimes when people are really, really depressed or even things like suicidal, they're actually really logical about it. And this is what's really challenging is if you try to talk to these people, you'll oftentimes find that their logic is actually really sound. And even in the cases when it's not really sound, you can try to convince them otherwise, but it seems like you sometimes hit a brick wall. And if you're struggling in this way, you may have noticed that you may try to argue against yourself, but your mind always comes to the same conclusion.


In [11]:
fps = cap.get(cv2.CAP_PROP_FPS)

In [12]:
fps

60.0

In [13]:
i=0
fps = cap.get(cv2.CAP_PROP_FPS)

# Read until video is completed
while(cap.isOpened()):
    if i % 60 != 0 and i <= (fps * result['segments'][-1]['end']):
        i+=1
        continue
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        scaled_raw_img = preprocess_frame(frame)
        analyse_frame(scaled_raw_img)
        
        # # Display the resulting frame
        plt.figure(figsize=(3,3))
        plt.imshow(frame)
        plt.axis('off')
        plt.show()

        print(i, end='\t')

    # Break the loop
    else:
        break
    i+=1

C:\Users\Ashwin-Rao\anaconda3\envs\music\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


NameError: name 'mode' is not defined

In [ ]:
from empath import Empath
lexicon = Empath()

In [ ]:
text_map = {'0': "The person is not depressed.", 
            '1':"The person is mildly depressed", 
            '2':"The person is severely depressed."}

empath_encoded_test = []

analysis = lexicon.analyze(result['text'], normalize=True)
array = [value for value in analysis.values()]
empath_encoded_test.append(array)

In [ ]:
text_model = joblib.load("text_classification.joblib")
y_pred = text_model.predict(empath_encoded_test)
print(text_map.get(str(y_pred[0])), '\n\n\n')

In [ ]:
print(text_map.get(str(y_pred[0])), '\n\n\n')

i = 0; results = []
for text in result['segments']:
    i += 60
    if predicted_values[i] == 0:
        print("Angry:", text['text'])
    elif predicted_values[i] == 1:
        print("Disgusted:", text['text'])
    elif predicted_values[i] == 2:
        print("Fearful:", text['text'])
    elif predicted_values[i] == 3:
        print("Happy:", text['text'])
    elif predicted_values[i] == 4:
        print("Neutral:", text['text'])
    elif predicted_values[i] == 5:
        print("Sad:", text['text'])
    elif predicted_values[i] == 6:
        print("Surprised:", text['text'])